In [141]:
import cv2 as cv
import numpy as np
import random

In [142]:
UBIT = 'pravi'
np.random.seed(sum([ord(a) for a in UBIT]))

# Extract SIFT features and draw the keypoints

In [143]:
def extract_sift(filepath,output_file):
    img = cv.imread(filepath)
    gray = cv.imread(filepath,0)
    sift = cv.xfeatures2d.SIFT_create()
    kp = sift.detect(gray,None)
    kpimg = cv.drawKeypoints(gray, kp, None, flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    cv.imwrite(output_file,kpimg)

In [144]:
extract_sift('mountain1.jpg','task1_sift1.jpg')

In [145]:
extract_sift('mountain2.jpg','task1_sift2.jpg')

# Good matches 

In [146]:
img1 = cv.imread('mountain1.jpg')
img2 = cv.imread('mountain2.jpg')
img1_gray = cv.imread('mountain1.jpg',0)
img2_gray = cv.imread('mountain2.jpg',0)

sift = cv.xfeatures2d.SIFT_create()
# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

# BFMatcher with default params
bf = cv.BFMatcher()
matches = bf.knnMatch(des1,des2, k=2)

# Apply ratio test
good = []
pts1 = []
pts2 = []
for m,n in matches:
    if m.distance < 0.75*n.distance:
        good.append(m)
        pts2.append(kp2[m.trainIdx].pt)
        pts1.append(kp1[m.queryIdx].pt)

# cv2.drawMatchesKnn expects list of lists as matches.
img3 = cv.drawMatches(img1,kp1,img2,kp2,good,None,flags=2)
cv.imwrite('task1_matches_knn.jpg',img3)

True

# Homography matrix

In [147]:
MIN_MATCH_COUNT = 10
if len(good)>MIN_MATCH_COUNT:
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

    M, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC,5.0)
    matchesMask = mask.ravel().tolist()

    h,w = img1_gray.shape
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv.perspectiveTransform(pts,M)

else:
    print ("Not enough matches are found",len(good),MIN_MATCH_COUNT)
    matchesMask = None

In [148]:
draw_params = dict(matchColor = (255,0,0), # draw matches in red color
                   singlePointColor = None,
                   matchesMask = matchesMask[:10], # draw only 10 inliers
                   flags = 2)

img3 = cv.drawMatches(img1,kp1,img2,kp2,good[:10],None,**draw_params)

In [149]:
cv.imwrite('Homography.jpg',img3)

True

In [150]:
#Print homography matrix
print(M)

[[ 1.58720376e+00 -2.91747553e-01 -3.95226519e+02]
 [ 4.48097764e-01  1.43063310e+00 -1.90273584e+02]
 [ 1.20808480e-03 -6.07787703e-05  1.00000000e+00]]


In [151]:
random_pt1 = []
random_pt2 = []
for i in [np.random.randint(0,len(good)-1) for x in range(10)]:
    random_pt1.append(good[i])
    random_pt2.append(matchesMask[i]) 
draw_params_2 = dict(matchColor = (0,255,0), # draw matches in red color
                   singlePointColor = None,
                   matchesMask = random_pt2, # draw only 10 inliers
                   flags = 2)

img4 = cv.drawMatches(img1,kp1,img2,kp2,random_pt1,None,**draw_params_2)
cv.imwrite('task1_matches.jpg',img4)

True

# Warp the two images

In [103]:
def warpImages(img1, img2, H):
    rows1, cols1 = img1.shape[:2]
    rows2, cols2 = img2.shape[:2]

    list_of_points_1 = np.float32([[0,0], [0,rows1], [cols1, rows1], [cols1,0]]).reshape(-1,1,2)
    temp_points = np.float32([[0,0], [0,rows2], [cols2, rows2], [cols2,0]]).reshape(-1,1,2)

    list_of_points_2 = cv.perspectiveTransform(temp_points, H)
    list_of_points = np.concatenate((list_of_points_1, list_of_points_2), axis=0)

    [x_min, y_min] = np.int32(list_of_points.min(axis=0).ravel() - 0.5)
    [x_max, y_max] = np.int32(list_of_points.max(axis=0).ravel() + 0.5)

    translation_dist = [-x_min, -y_min]
    H_translation = np.array([[1, 0, translation_dist[0]], [0, 1, translation_dist[1]], [0,0,1]])

    output_img = cv.warpPerspective(img2, H_translation.dot(H), (x_max - x_min, y_max - y_min))
    output_img[translation_dist[1]:rows1+translation_dist[1],translation_dist[0]:cols1+translation_dist[0]] = img1
    return output_img

In [105]:
result = warpImages(img2,img1,M)
cv.imwrite('task1_pano.jpg',result)

True